Chapter 10: Data Aggregation and Group Operations

In [2]:
import numpy as np
import pandas as pd

Group operations follow a split-apply-combine sequence. First data is split into groups based on one or more keys (either split across rows or columns), then a function is applied to each group, then the results are combined into a result.

In [3]:
# Start with a tabular dataset made into a DataFrame
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.768734,0.689586
1,a,2,0.937011,-1.163331
2,None,1,0.319090,1.455575
3,b,2,-0.089813,-0.655382
4,b,1,-0.070830,0.392339
5,a,<NA>,-0.521848,-1.527677
6,None,1,-1.050236,-0.305656


In [4]:
# Sort the df by picking out the data1 column and group by key1
grouped = df["data1"]
grouped

0    0.768734
1    0.937011
2    0.319090
3   -0.089813
4   -0.070830
5   -0.521848
6   -1.050236
Name: data1, dtype: float64

In [5]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [6]:
# The groupby object has all the info needed to apply an operation to each of the groups
grouped.mean()

key1
a    0.394632
b   -0.080322
Name: data1, dtype: float64

In [7]:
# Passing multiple arrays to "group by" gives different groups and different values
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       0.768734
      2       0.937011
b     1      -0.070830
      2      -0.089813
Name: data1, dtype: float64

In [8]:
# You can group by any arrays of the correct length
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005    0.207582
    2006    0.319090
OH  2005    0.339461
    2006   -0.560533
Name: data1, dtype: float64

In [9]:
# size returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [10]:
df.groupby("key1").size()

key1
a    3
b    2
dtype: int64

In [11]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [12]:
# The object returned by groupby supports iteration
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.768734  0.689586
1    a     2  0.937011 -1.163331
5    a  <NA> -0.521848 -1.527677
b
  key1  key2     data1     data2
3    b     2 -0.089813 -0.655382
4    b     1 -0.070830  0.392339


In [13]:
# This allows for the computation of a dictionary of the data pieces as a one-liner:
pieces = {name: group for name, group in df.groupby("key1")}
pieces

{'a':   key1  key2     data1     data2
 0    a     1  0.768734  0.689586
 1    a     2  0.937011 -1.163331
 5    a  <NA> -0.521848 -1.527677,
 'b':   key1  key2     data1     data2
 3    b     2 -0.089813 -0.655382
 4    b     1 -0.070830  0.392339}

In [14]:
# We can group the columns of the dataframe by whether they start with "key" or "data"
grouped = df.groupby({"key1": "key", "key2" : "key", "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  0.768734  0.689586
1  0.937011 -1.163331
2  0.319090  1.455575
3 -0.089813 -0.655382
4 -0.070830  0.392339
5 -0.521848 -1.527677
6 -1.050236 -0.305656
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [15]:
# Compute the mean only for the data2 column and return the result as a dataframe
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.689586
     2    -1.163331
b    1     0.392339
     2    -0.655382

In [18]:
# Grouping with Dictionaries and Series
# make new df
people = pd.DataFrame(np.random.standard_normal((5, 5)), columns=["a", "b", "c", "d", "e"], index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-1.058363,0.328541,0.321652,-0.924231,1.716669
Steve,-0.735919,-0.723418,2.008609,1.060049,-1.258358
Wanda,0.546722,-0.747032,0.777247,0.078575,0.909879
Jill,-0.578510,-1.187899,-0.684946,0.149758,-1.038906
Trey,0.004153,0.234222,-2.504285,-0.594059,-0.663711


In [22]:
# add NA values
people.iloc[2:4, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-1.058363,0.328541,0.321652,-0.924231,1.716669
Steve,-0.735919,-0.723418,2.008609,1.060049,-1.258358
Wanda,0.546722,NaN,NaN,0.078575,0.909879
Jill,-0.578510,NaN,NaN,0.149758,-1.038906
Trey,0.004153,0.234222,-2.504285,-0.594059,-0.663711


In [24]:
# Create a dictionary containing a mapping
mapping = {"a": "red", "b": "red", "c": "blue", "d": "blue", "e": "red", "f": "orange"}
mapping
# The mapping "f" above will get ignored

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [25]:
# Pass the dictionary to "groupby"
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,-0.602579,0.986847
Steve,3.068658,-2.717695
Wanda,0.078575,1.456601
Jill,0.149758,-1.617416
Trey,-3.098343,-0.425337


In [26]:
# We can also pass the dictionary to a series
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [27]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,1,2
Trey,2,3


In [28]:
# Grouping with Functions

# Group the previous dataframe by passing the len function to groupby
people.groupby(len).sum()

,a,b,c,d,e
3,-1.058363,0.328541,0.321652,-0.924231,1.716669
4,-0.574358,0.234222,-2.504285,-0.444301,-1.702617
5,-0.189198,-0.723418,2.008609,1.138624,-0.348479


In [29]:
# You can also pass a mixed function/array/dict/Series to groupby
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.058363,0.328541,0.321652,-0.924231,1.716669
4,two,-0.578510,0.234222,-2.504285,-0.594059,-1.038906
5,one,-0.735919,-0.723418,2.008609,0.078575,-1.258358


In [30]:
# Data Aggregation
# An aggregation is a data transformation which produces a scalar from an array (i.e. mean, count, min, sum)
df

,key1,key2,data1,data2
0,a,1,0.768734,0.689586
1,a,2,0.937011,-1.163331
2,None,1,0.319090,1.455575
3,b,2,-0.089813,-0.655382
4,b,1,-0.070830,0.392339
5,a,<NA>,-0.521848,-1.527677
6,None,1,-1.050236,-0.305656


In [31]:
# Group by key1
grouped = df.groupby("key1")
# Find 2 smallest values in the grouped df under "data1" column
grouped["data1"].nsmallest(2)

key1   
a     5   -0.521848
      0    0.768734
b     3   -0.089813
      4   -0.070830
Name: data1, dtype: float64

In [32]:
# You can define your own functions which act on the whole array and pass them to agg
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.458858,2.217263
b,1,0.018983,1.047721


In [36]:
# Return to tipping dataset
tips = pd.read_csv("datasets/tips.csv")
print("Shape of data frame: {}".format(tips.shape))
tips.head()

Shape of data frame: (244, 6)


,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [37]:
# Add a tip percentage column
tips["tip_pct"] = tips["tip"]/tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [39]:
# Group the tips by day and smoking status
grouped = tips.groupby(["day", "smoker"])
# Pull the tip percentage column
grouped_pct = grouped["tip_pct"]
# Pass the name of the function as a string to agg
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [40]:
# Can also pass a list of functions
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [41]:
# Can give the returns from the functions new names
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [43]:
# For data frames, can pass the same function to different columns or different functions to different columns
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [44]:
# To apply different functions to different columns, pass a dictionary to agg
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [45]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"], "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [46]:
# Can change the index from grouped keys to a flattened index
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [47]:
# The apply method splits the object it acts on into pieces, invokes a function on each piece, and then concatenates the pieces together.
# Here we want to select the top five tip percentage values by group

# Define a function
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [48]:
# Now group by "smoker" and call apply with the top function
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [49]:
# Functions with optional inputs can also be passed
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [50]:
# Again, you can flatten the indexes
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [51]:
# pandas makes it easy to do quantile analysis on a dataset
# Make new dataset
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,1.027931,0.165492
1,0.210694,1.232825
2,1.119653,0.767350
3,-0.939892,0.642919
4,-1.875444,-0.047158


In [52]:
# Now find the quartiles for the "data1" column
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0      (0.257, 1.801]
1     (-1.287, 0.257]
2      (0.257, 1.801]
3     (-1.287, 0.257]
4    (-2.838, -1.287]
5     (-1.287, 0.257]
6     (-1.287, 0.257]
7     (-1.287, 0.257]
8      (0.257, 1.801]
9      (0.257, 1.801]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.838, -1.287] < (-1.287, 0.257] < (0.257, 1.801] < (1.801, 3.345]]

In [53]:
# We can then pass the quartiles object to the groupby function
def get_stats(group):
    return pd.DataFrame({"min": group.min(), "max": group.max(), "count": group.count(), "mean": group.mean()})

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-2.838, -1.287] data1 -2.831376 -1.309342     94 -1.772795
                 data2 -2.236223  1.980534     94  0.180596
(-1.287, 0.257]  data1 -1.286174  0.256048    489 -0.439263
                 data2 -2.648368  3.390833    489 -0.054720
(0.257, 1.801]   data1  0.264530  1.767108    381  0.851616
                 data2 -2.540955  3.417759    381  0.086354
(1.801, 3.345]   data1  1.802152  3.345291     36  2.186267
                 data2 -3.352918  2.382029     36 -0.020267

In [54]:
# But note we can do that more easily
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-2.838, -1.287] -2.831376 -1.309342    94 -1.772795 -2.236223  1.980534   
(-1.287, 0.257]  -1.286174  0.256048   489 -0.439263 -2.648368  3.390833   
(0.257, 1.801]    0.264530  1.767108   381  0.851616 -2.540955  3.417759   
(1.801, 3.345]    1.802152  3.345291    36  2.186267 -3.352918  2.382029   

                                  
                 count      mean  
data1                             
(-2.838, -1.287]    94  0.180596  
(-1.287, 0.257]    489 -0.054720  
(0.257, 1.801]     381  0.086354  
(1.801, 3.345]      36 -0.020267

In [55]:
# cut returns equal length buckets, qcut returns equal sized buckets
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    3
1    2
2    3
3    0
4    0
Name: data1, dtype: int64

In [56]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.831376 -0.694713    250 -1.265224
      data2 -2.648368  2.410464    250 -0.027802
1     data1 -0.691476  0.085769    250 -0.314050
      data2 -2.513231  3.390833    250 -0.031128
2     data1  0.085866  0.707041    250  0.381621
      data2 -2.540955  3.417759    250  0.113620
3     data1  0.712905  3.345291    250  1.284570
      data2 -3.352918  2.739433    250  0.034867